In [48]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta

In [5]:
def get_stats(driver, url):
    driver.implicitly_wait(random.randint(0, 6))
    driver.get(url)
    button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
    button.click()
    div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
    t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
          if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
    score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
    t['score'] = [score[0], score[2]]
    posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
    t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
    game_stats = {}
    for name, stats in t.items():
        if '(' in ''.join(stats):
            s = stats[0].split(' ') + stats[1].split(' ')
            if len(s) == 4:
                s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                game_stats[name] = [s[0], s[2]]
                game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
            else:
                s = [i for i in s if '%' not in i]
                game_stats[name] = s
        else:
            game_stats[name] = stats
    return game_stats

In [6]:
get_stats(webdriver.Chrome(), 'https://www.fotmob.com/match/3916934/matchfacts/standard-liege-vs-gent')

{'Expected goals (xG)': ['1.08', '2.30'],
 'Total shots': ['5', '22'],
 'Big chances': ['1', '2'],
 'Big chances missed': ['0', '0'],
 'Accurate passes': ['176', '411'],
 'Accurate passes percentage': [0.72, 0.86],
 'Fouls committed': ['12', '11'],
 'Offsides': ['1', '0'],
 'Corners': ['2', '8'],
 'Shots off target': ['2', '10'],
 'Shots on target': ['2', '8'],
 'Blocked shots': ['1', '4'],
 'Hit woodwork': ['0', '1'],
 'Shots inside box': ['2', '13'],
 'Shots outside box': ['3', '9'],
 'xG first half': ['0.94', '1.25'],
 'xG second half': ['0.14', '1.05'],
 'xG open play': ['0.29', '1.93'],
 'xG set play': ['0.00', '0.37'],
 'xG penalty': ['0.79', '0.00'],
 'xG on target (xGOT)': ['1.16', '1.02'],
 'Passes': ['244', '478'],
 'Own half': ['115', '157'],
 'Opposition half': ['61', '254'],
 'Accurate long balls': ['21', '42'],
 'Accurate long balls percentage': [0.44, 0.74],
 'Accurate crosses': ['1', '6'],
 'Accurate crosses percentage': [0.17, 0.33],
 'Throws': ['15', '23'],
 'Yellow c

In [49]:
driver = webdriver.Chrome()
driver.implicitly_wait(3)
driver.get('https://www.fotmob.com/?filter=&show=all&q=&date=20220806')
divs = driver.find_elements(By.XPATH, '//div[@class="css-1akrvvq-Group e7pc1841"]')
xg_leagues = ['UNITED STATES - MAJOR LEAGUE SOCCER', 'ENGLAND - PREMIER LEAGUE', 'GERMANY - BUNDESLIGA', 'FRANCE - LIGUE 1',
    'AUSTRIA - BUNDESLIGA', 'PORTUGAL - LIGA PORTUGAL', 'BELGIUM - FIRST DIVISION A', 'SWITZERLAND - SUPER LEAGUE']
league = {}
for count, bucket in enumerate(divs):
    league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
    if league_name not in xg_leagues:
        continue
    url = bucket.find_elements(By.TAG_NAME, 'a')
    urls = [u.get_attribute('href') for u in url]
    time = bucket.text
    time = [datetime.strptime(i, '%H:%M') + timedelta(minutes=50) for i in time.split('\n') if ':' in i]
    urls = urls[1:]
    league[league_name] = list(zip(urls, time))

In [51]:
league

{'UNITED STATES - MAJOR LEAGUE SOCCER': [('https://www.fotmob.com/match/3787558/matchfacts/atlanta-united-vs-seattle-sounders-fc',
   datetime.datetime(1900, 1, 1, 15, 50)),
  ('https://www.fotmob.com/match/3787557/matchfacts/charlotte-fc-vs-chicago-fire-fc',
   datetime.datetime(1900, 1, 1, 19, 50)),
  ('https://www.fotmob.com/match/3787698/matchfacts/cf-montreal-vs-inter-miami-cf',
   datetime.datetime(1900, 1, 1, 20, 20)),
  ('https://www.fotmob.com/match/3787567/matchfacts/columbus-crew-vs-new-york-city-fc',
   datetime.datetime(1900, 1, 1, 20, 20)),
  ('https://www.fotmob.com/match/3787588/matchfacts/dc-united-vs-new-york-red-bulls',
   datetime.datetime(1900, 1, 1, 20, 20)),
  ('https://www.fotmob.com/match/3787566/matchfacts/fc-cincinnati-vs-philadelphia-union',
   datetime.datetime(1900, 1, 1, 20, 20)),
  ('https://www.fotmob.com/match/3787589/matchfacts/orlando-city-vs-new-england-revolution',
   datetime.datetime(1900, 1, 1, 20, 20)),
  ('https://www.fotmob.com/match/3787697/